In [130]:
from datetime import datetime
import requests
import polyline
from xml.dom import minidom

In [131]:
# name of the file with sql query result (without the extension)
file_name = "location_data_02"

In [132]:
# parsing the query data
location_list = []
with open(file_name + ".txt") as fin:
    line = fin.readline()
    count = 1
    while line:
        if (count > 3 and len(line.split("|")) > 1):
            line = line.split("|")
            location_list.append([int(line[1].strip())/1000, float(line[2].strip()), float(line[3].strip())])
        line = fin.readline()
        count += 1
location_list

[[1516574062.921, 47.681201907234, -122.1115379675],
 [1516574280.096, 47.681218333193, -122.11155587631],
 [1516574880.002, 47.681218333193, -122.11155587631],
 [1516575480.002, 47.681218333193, -122.11155587631],
 [1516576080.001, 47.681218333193, -122.11155587631],
 [1516576680.002, 47.681218333193, -122.11155587631],
 [1516577280.001, 47.681218333193, -122.11155587631],
 [1516577879.968, 47.681218333193, -122.11155587631],
 [1516578480.001, 47.681218333193, -122.11155587631],
 [1516579079.951, 47.681218333193, -122.11155587631],
 [1516579679.941, 47.681218333193, -122.11155587631],
 [1516580279.969, 47.681218333193, -122.11155587631],
 [1516580879.921, 47.681218333193, -122.11155587631],
 [1516581479.858, 47.681218333193, -122.11155587631],
 [1516582079.926, 47.681218333193, -122.11155587631],
 [1516582679.89, 47.681218333193, -122.11155587631],
 [1516583279.837, 47.681218333193, -122.11155587631],
 [1516583879.855, 47.681218333193, -122.11155587631],
 [1516584479.829, 47.681218333

In [133]:
# writing the parsed sql query data to a .gpx file
gpx = open(file_name + ".gpx", 'w')
gpx.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
gpx.write("<gpx>\n")
gpx.write("<trk>\n")
gpx.write("<trkseg>\n")
for location in location_list:
    gpx.write("<trkpt lat=\"" + str(location[1]) + "\" lon=\"" + str(location[2]) + "\">\n")
    gpx.write("<time>" + str(datetime.fromtimestamp(location[0]).isoformat()) +"</time>\n")
    gpx.write("</trkpt>\n")
gpx.write("</trkseg>\n")
gpx.write("</trk>\n")
gpx.write("</gpx>\n")
gpx.close()

In [134]:
# calling the osrm api with the coordinates
url = "http://127.0.0.1:5000/match/v1/drive/"
for i in location_list:
    url += str(i[2]) + "," + str(i[1]) + ";"

In [135]:
url = url[0:len(url)-1]
url += "?tidy=true"

In [136]:
r = requests.get(url)
res = r.json()
res

{'code': 'Ok',
 'matchings': [{'confidence': 0.033415,
   'geometry': '}n~aH`{hhV?^?dJ?zBbE@xD@tB}DzEy@zAwH`DcP|HzElB|EbBlOBzt@@pI?`INxHn@pHrAzGlBbGdCdFbD|DtDlCxDlB|DdB|DbB~D~AdEfBhEhBlEdBvE`AvE`@|EDxEYvE{@pEwAvE_BpEeBhE_BrEaB~E{A|Eq@tE?tD`@zD|@nD~AlD~B`DxClCnDfCzExBrEbCbFjCrFjCpFhCnFhCnFjCnFbCjFvBjFjB|FnAjGv@xGd@bHJpHO~GYpG[vG_@dI]hH@fIPfGVxHBbIe@lHcAbHaA`H{@dG_AnG{@`Gm@`G]lGEnGFnGLpGAnGm@pGgBhFmD`BYyCvBcBtDTtD\\dEd@pDVl_AdBdJ~IsUTy@mH',
   'legs': [{'steps': [],
     'distance': 12.1,
     'duration': 1.8,
     'summary': '',
     'weight': 1.8},
    {'steps': [],
     'distance': 133.6,
     'duration': 19.1,
     'summary': '',
     'weight': 19.1},
    {'steps': [],
     'distance': 156.3,
     'duration': 27.9,
     'summary': '',
     'weight': 27.9},
    {'steps': [],
     'distance': 202.8,
     'duration': 27.3,
     'summary': '',
     'weight': 27.3},
    {'steps': [],
     'distance': 145.3,
     'duration': 21.5,
     'summary': '',
     'weight': 21.5},
    {'steps': [],

In [137]:
decoded_coordinates = []
for item in res['matchings']:
    decoded_coordinates.append(polyline.decode(item['geometry']))
decoded_coordinates = [item for sublist in decoded_coordinates for item in sublist]

In [138]:
print(len(decoded_coordinates))

484


In [139]:
# writing the result of the api call to a .gpx file
dgpx = open(file_name + "_decoded.gpx", 'w')
dgpx.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
dgpx.write("<gpx>\n")
dgpx.write("<trk>\n")
dgpx.write("<trkseg>\n")
for location in decoded_coordinates:
    dgpx.write("<trkpt lat=\"" + str(location[0]) + "\" lon=\"" + str(location[1]) + "\">\n")
    dgpx.write("</trkpt>\n")
dgpx.write("</trkseg>\n")
dgpx.write("</trk>\n")
dgpx.write("</gpx>\n")
dgpx.close()

In [140]:
# CODE FOR THE Deaf_Bind_Service_Center file parsing

# Code to parse GPX files and request snapping from osrm api and then write the result back to a .gpx file
xmldoc = minidom.parse('Deaf_Bind_Service_Center.gpx')
itemlist = xmldoc.getElementsByTagName('trkpt')
url = "http://127.0.0.1:5000/match/v1/drive/"
for pt in itemlist:
    url += pt.attributes['lon'].value + ","+  pt.attributes['lat'].value +";"
url = url[0:len(url)-1]
url += "?tidy=true"
r = requests.get(url)
res = r.json()
decoded_coordinates = []

for item in res['matchings']:
    decoded_coordinates.append(polyline.decode(item['geometry']))
decoded_coordinates = [item for sublist in decoded_coordinates for item in sublist]


dgpx = open("Deaf_Bind_Service_Center_decoded.gpx", 'w')
dgpx.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
dgpx.write("<gpx>\n")
dgpx.write("<trk>\n")
dgpx.write("<trkseg>\n")
print(len(decoded_coordinates))
for location in decoded_coordinates:
    dgpx.write("<trkpt lat=\"" + str(location[0]) + "\" lon=\"" + str(location[1]) + "\">\n")
    dgpx.write("</trkpt>\n")
dgpx.write("</trkseg>\n")
dgpx.write("</trk>\n")
dgpx.write("</gpx>\n")
dgpx.close()

121
